# Gathering Data

In [24]:
from functions import *
from scivol import *
from helpers import *
import nibabel as nib
import ipyvolume as ipv
import numpy as np
import json
#import nilearn
import ants

In [25]:
proj_root = parent_directory()
print(proj_root)
mri_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
template_folder = "/Users/joachimpfefferkorn/repos/neurovolume/templates"
output_folder = os.path.join(proj_root, "output/")

default_cmap = 'nipy_spectral'

/Users/joachimpfefferkorn/repos/neurovolume


In [26]:
#anat_img = nib.load(mri_input_filepath)
raw_anat_img = ants.image_read(mri_input_filepath)

In [34]:
print(raw_anat_img)
explore_3D_array(raw_anat_img.numpy(), cmap=default_cmap)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

# Skull Stripping
Using ANTs

[this notebook](https://github.com/Angeluz-07/MRI-preprocessing-techniques/blob/main/notebooks/09_brain_extraction_with_template.ipynb) is what we're basing this on

## Load Template and Mask

In [28]:
mni_template_img = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_template_mask = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")

In [ ]:
print(mni_template_img)
explore_3D_array(mni_template_img.numpy(), cmap=default_cmap)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (197, 233, 189)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, 134.0, -72.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



interactive(children=(IntSlider(value=98, description='SLICE', max=196), Output()), _dom_classes=('widget-inte…

In [30]:
print(mni_template_mask)
explore_3D_array(mni_template_mask.numpy(), cmap=default_cmap)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (197, 233, 189)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, 134.0, -72.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



interactive(children=(IntSlider(value=98, description='SLICE', max=196), Output()), _dom_classes=('widget-inte…

## Register Template and Masks to Raw image
Here we warp the MNI template

In [50]:
transformation = ants.registration(
    fixed=raw_anat_img,
    moving=mni_template_img, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [0x15b777ca8,0x15b777e88,1] -m mattes[0x15b777ca8,0x15b777e88,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x15b777ca8,0x15b777e88,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpcii1r2fv,0x15b502ec8,0x15bcc3c68] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
All_Command_lines_OK
Using single precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Center of mass alignment using fixed image: 0x15b777ca8 and moving image: 0x15b777e88 (type = Euler3DTransform)
  Reading mask(s).
    Registration stage 0
      No fixed mask
      No moving mask
    Registration stage 1
      No fixed mask
      No moving mask
  number of levels = 4
  number of levels = 3
  fixed image: 0x15b777ca8
  moving image: 0x15b777e88
  fixed image: 0x15b777ca8
  moving image: 0x15b777e88

In [51]:
registered_template_img = transformation['warpedmovout']

The following comparison function asserts that the new `registered_template_img` image has been transformed to the size and shape of the `raw_anat_img`

In [49]:
explore_3D_array_comparison(
    arr_before=raw_anat_img.numpy(), 
    arr_after=registered_template_img.numpy()
)
#I don't love "before" and "after" as the names here as we are really only changing the template
# This is pretty confusing, so just ignore the names for now
# Maybe I'll change this and write my own helper functions

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

Next, we apply this to the mask so that it fits over the brain in the data we wish to visualize

In [52]:
brain_mask = ants.apply_transforms(
    fixed=transformation['warpedmovout'],
    moving=mni_template_mask,
    transformlist=transformation['fwdtransforms'],
    interpolator='nearestNeighbor',
    verbose=True
)

['-d', '3', '-i', '0x15b88f948', '-o', '0x15b49c588', '-r', '0x15b49fae8', '-n', 'nearestNeighbor', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpcii1r2fv1Warp.nii.gz', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpcii1r2fv0GenericAffine.mat']
Using double precision for computations.
Input scalar image: 0x15b88f948
Could not create ImageIO for the input file, assuming dimension = 3 and scalar pixel type
Reference image: 0x15b49fae8
The composite transform comprises the following transforms (in order): 
  1. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpcii1r2fv0GenericAffine.mat (type = AffineTransform)
  2. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpcii1r2fv1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpolateImageFunction
Output warped image: 0x15b49c588


In [53]:
explore_3D_array_with_mask_contour(raw_anat_img.numpy(), brain_mask.numpy())

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

Let's fine tune the mask a bit

In [54]:
brain_mask_dilated = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')

In [ ]:
explore_3D_array_with_mask_contour(raw_anat_img.numpy(), brain_mask_dilated.numpy())

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

# Mask the Image and Create Sections

In [56]:
brain_anat = ants.mask_image(raw_anat_img, brain_mask_dilated)

In [58]:
explore_3D_array(brain_anat.numpy())

interactive(children=(IntSlider(value=255, description='SLICE', max=511), Output()), _dom_classes=('widget-int…

In [59]:
print(brain_anat)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 296)
	 Spacing    : (0.4785, 0.4785, 0.5)
	 Origin     : (119.989, 104.52, -84.2457)
	 Direction  : [-1.      0.0025  0.     -0.0025 -1.      0.      0.      0.      1.    ]



# Building Scivol

It looks like ANTS already applies the affine (double check this), so I think you can just 86 that from your workflow

In [62]:
ipv.quickvolshow(brain_anat.numpy())

/Users/joachimpfefferkorn/repos/neurovolume/.venv/lib/python3.11/site-packages/ipyvolume/serialize.py:102: RuntimeWarning: invalid value encountered in cast
  subdata[..., i] = ((gradient[i][zindex] / 2.0 + 0.5) * 255).astype(np.uint8)


Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [63]:
affine = np.eye(4) #TODO get rid of the affines in general as ANTS doesn't need them

Initialize the scivol object

In [65]:
skullstrip = Scivol("skullstrip", affine, 0.2)

Create and add the grids

In [66]:
skullstrip.add_grids([Grid("full_anat",[raw_anat_img.numpy()]),
Grid("brain_anat", [brain_anat.numpy()])])

writing full_anat to grid
writing brain_anat to grid


In [67]:
skullstrip.save_scivol(output_folder)

saving skullstrip.scivol
writing skullstrip.scivol


Before moving to blender to render these images, let's make sure that we can parse these scivol files here

In [68]:
scivol_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/skullstrip.scivol"
with open(scivol_path, 'r') as file:
    data = json.load(file)
print("scivol name: ", data['name'], "\n")
for grid_key in data['grids'].keys():
    print(grid_key)
    vol = np.array(data['grids'][grid_key]['frames'][0])

scivol name:  skullstrip 

full_anat
brain_anat


In [69]:
print(data['tolerance'])
print((np.asarray(data['affine'])))

0.2
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [70]:
print(data['grids'].keys())
for grid in data['grids']:
    print(grid)

dict_keys(['full_anat', 'brain_anat'])
full_anat
brain_anat


Cool, let's quickly visualize this

Wow, that is taking a while to visualize. I wonder what's up?